In [1]:
from bs4 import BeautifulSoup
import requests
import time, os

In [27]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [24]:
import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_2/py')

In [25]:
from loadpage import *

In [5]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.fictiondb.com/search/search.htm")

In [11]:
search_bar = driver.find_element_by_xpath("//input[@name='author'][@type='text']")

In [12]:
search_bar.clear()
search_bar.send_keys("Jane Austen")
search_bar.send_keys(Keys.RETURN)

In [28]:
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='btn btn-primary' and @id='YesBtn']"))).click()

TimeoutException: Message: 


In [32]:
elem = driver.find_element_by_link_text("Austen, Jane")
elem.click()

In [33]:
elem = driver.find_element_by_class_name("btn.btn-secondary")
elem.click()

In [19]:
url = driver.current_url
print(url)

https://www.fictiondb.com/author/jane-austen~293.htm


In [36]:
soup = get_soup("https://www.fictiondb.com/author/author.php?authorid=293&ltyp=3")

In [35]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xmlns:v="urn:schemas-microsoft-com:vml">
<head>
<title>Error.</title>
<meta content="application/xhtml+xml; charset=utf-8" http-equiv="Content-Type"/>
<meta content="index, follow" name="robots"/>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
</head>
<body>
<div id="header">
<div id="logo">
<a href="../"><img alt="FictionDB Logo" src="../images/fictiondb_logo.jpg"/></a>
</div>
</div>
<div class="colmask fullpage">
<div class="col1">
<!-- Column 1 start -->
<h1>You have reached a page that does not exist</h1>
<p>Click on the logo above to return to FictionDB</p>
<!-- Column 1 end -->
</div>
</div>
</body>
</html>

In [41]:
driver.find_element_by_xpath("//a[@itemprop='datePublished']").getText();

AttributeError: 'WebElement' object has no attribute 'getText'

In [46]:
years = driver.find_elements_by_xpath("//a[@itemprop='datePublished']")

In [47]:
[s.text for s in years]

['1818',
 '1817',
 '1815',
 '1814',
 '1813',
 '1811',
 '1805',
 '1805',
 '1803',
 '1790',
 '1786']

### Test actor on imdb

In [247]:
import numpy as np
import pandas as pd
def get_movie_brief_actor(actor, soup):
    """
    Getting brief data from individual movie webpage (for actor dictionary)
    """
    
    headers=['actor','title','year','rating','vote','genre_list','budget','opening','gross_usa',\
             'gross_cw','runtime','director','writer','star','distributor']
    
    # actor of interest
    actor = actor
    
    # find movie title
    title = " ".join(soup.find('h1').text.split()[:-1])
    
    
    # find rating
    rating = np.nan
    try:
        rating = float(soup.find('span',attrs={'itemprop':'ratingValue'}).text)
    except:
        pass
    
    # find vote (rating count)
    vote = np.nan
    try:
        vote = int(soup.find('span',attrs={'itemprop':'ratingCount'}).text.replace(',',''))
    except:
        pass
    
    
    # find list of genre
    genre_list=[]
    try:
        for genres in soup.find('div', class_="subtext").find_all('a')[:-1]:
            genre_list.append(genres.text)
    except:
        pass
    
        
    # find release date
    date = np.nan
    try:
        date_pre = soup.find('div', class_="subtext").find_all('a')[-1].text.split('(')[0]
        date = pd.to_datetime(date_pre) ## why is it Timestamp? format ='%d-%B-%Y'
    except:
        pass

    
#     # find metascorre
#     if soup.find('div',class_="metacriticScore score_favorable titleReviewBarSubItem") is not None:
#         meta = int(soup.find('div',class_="metacriticScore score_favorable titleReviewBarSubItem").text.strip('\n'))
#     else:
#         meta = np.nan
        
        
#     # find plot keywords
#     keyword_list=[]
#     for keywords in soup.find('div', class_="article", id="titleStoryLine").\
#     find('div', class_="see-more inline canwrap").find_all('a')[:-1]:
#         keyword_list.append(keywords.text.strip(' '))
        
    
    
    # find budget, opening weekend USA, gross USA, cumulative worldwide gross
    # assign default value:
    budget, opening, gross_usa, gross_cw, distributor = np.nan, np.nan, np.nan, np.nan, np.nan
    try:
        for line in soup.find('div', class_="article", id="titleDetails").find_all('h4'):        
            if "Budget:" in line:
                budget = int(''.join(s for s in line.next_sibling if s.isdigit()))
            if "Opening Weekend USA:" in line:
                opening = int(''.join(s for s in line.next_sibling if s.isdigit()))
            if "Gross USA:" in line:
                gross_usa = int(''.join(s for s in line.next_sibling if s.isdigit()))
            if "Cumulative Worldwide Gross:" in line:
                gross_cw = int(''.join(s for s in line.next_sibling if s.isdigit()))
            if "Production Co:" in line:
                distributor = line.findNext().text.replace(' ','')
    except:
        pass

        
    # find runtime
    runtime = np.nan
    try:
        runtime = int(soup.find_all('time')[-1].text.strip(' min'))
    except:
        pass
        
        
        
        
    # find director
    director= np.nan
    try:
        director = soup.find('div',class_="credit_summary_item").find('a').text
        link_d = soup.find('div',class_="credit_summary_item").find('a').get('href')
    except:
        pass
    
    # find writer
    writer = np.nan
    try:
        writer_line = soup.find_all('div',class_="credit_summary_item")[1].find_all('a')
        link_w = [w.get('href') for w in writer_line]
        writer = [w.text for w in writer_line]
        if '1 more credit' in writer:
            writer.remove('1 more credit')
            link_w.pop()
    except:
        pass
    
        
    # find star
    star = np.nan
    try:
        star_line = soup.find_all('div',class_="credit_summary_item")[2].find_all('a')
        link_s = [s.get('href') for s in star_line]
        star = [s.text for s in star_line]
        if 'See full cast & crew' in star:
            star.remove('See full cast & crew')
            link_s.pop()
    except:
        pass
    
    
    
#     # find language
#     language= np.nan
#     t= []
#     matching = []
#     for div in soup.find('div', class_="article", id="titleDetails").find_all('div'):
#         t.append(div.text.replace('\n','').replace(' ',''))
    
#     matching = [s for s in t if 'Language:' in s]
#     language = matching[0].replace(':',' ').replace('|',' ').split(' ')[1:]
    
    
#     # find country
#     country= np.nan
#     t= []
#     matching = []
#     for div in soup.find('div', class_="article", id="titleDetails").find_all('div'):
#         t.append(div.text.replace('\n','').replace(' ',''))
    
#     matching = [s for s in t if 'Country:' in s]
#     country = matching[0].replace(':',' ').replace('|',' ').split(' ')[1:]
    
        
    movie_dict = dict(zip(headers, [actor,
                                    title,
                                    date,
                                    rating,
                                    vote,
                                   genre_list,
                                   budget,
                                   opening,
                                   gross_usa,
                                   gross_cw,
                                   runtime,
                                    director,
                                    writer,
                                    star,
                                   distributor]))
    
    return movie_dict

In [3]:
movie_df = pd.read_pickle('../dump/movie_data')

In [4]:
movie_df

,movie_title,rating,vote,certificate,genre,release_date,metascore,keywords,budget,opening_weekend_usa,...,runtime,director,link_d,writer,link_w,star,link_s,distributor,language,country
0,After We Collided,5.0,16457,R,"[Drama, Romance]",2020-10-23,NaN,"[sequel, birthday, boyfriend girlfriend sex, b...",NaN,422899.0,...,105.0,Roger Kumble,/name/nm0474955/,"[Anna Todd, Mario Celaya]","[/name/nm6849771/, /name/nm8800659/]","[Josephine Langford, Hero Fiennes Tiffin, Dyla...","[/name/nm6466214/, /name/nm2842005/, /name/nm0...",CalMaple,"[English, Chinese]",[USA]
1,The Godfather,9.2,1614204,R,"[Crime, Drama]",1972-03-24,100.0,"[crime family, mafia, patriarch, organized cri...",6000000.0,302393.0,...,175.0,Francis Ford Coppola,/name/nm0000338/,"[Mario Puzo, Francis Ford Coppola]","[/name/nm0701374/, /name/nm0000338/]","[Marlon Brando, Al Pacino, James Caan]","[/name/nm0000008/, /name/nm0000199/, /name/nm0...",ParamountPictures,"[English, Italian, Latin]",[USA]
2,Harry Potter and the Sorcerer's Stone,7.6,654936,PG,"[Adventure, Family, Fantasy]",2001-11-16,64.0,"[magic, wizard, school of magic, based on nove...",125000000.0,90294621.0,...,159.0,Chris Columbus,/name/nm0001060/,"[J.K. Rowling, Steve Kloves]","[/name/nm0746830/, /name/nm0460141/]","[Daniel Radcliffe, Rupert Grint, Richard Harris]","[/name/nm0705356/, /name/nm0342488/, /name/nm0...",WarnerBros.,[English],"[UK, USA]"
3,Unknown,6.8,248673,PG-13,"[Action, Thriller]",2011-02-18,NaN,"[amnesia, coma, airport, scientist, nurse]",30000000.0,21856389.0,...,113.0,Jaume Collet-Serra,/name/nm1429471/,"[Oliver Butcher, Stephen Cornwell]","[/name/nm0124700/, /name/nm0180508/]","[Liam Neeson, Diane Kruger, January Jones]","[/name/nm0000553/, /name/nm1208167/, /name/nm0...",DarkCastleEntertainment,"[English, German, Turkish, Arabic]","[UK, Germany, France, USA]"
4,The Lord of the Rings: The Fellowship of the Ring,8.8,1656253,PG-13,"[Action, Adventure, Drama]",2001-12-19,92.0,"[ring, quest, hobbit, epic, elf]",93000000.0,47211490.0,...,171.0,Peter Jackson,/name/nm0001392/,"[J.R.R. Tolkien, Fran Walsh, 2 more credits]","[/name/nm0866058/, /name/nm0909638/, fullcredi...","[Elijah Wood, Ian McKellen, Orlando Bloom]","[/name/nm0000704/, /name/nm0005212/, /name/nm0...",NewLineCinema,"[English, Sindarin]","[NewZealand, USA]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,From the Terrace,6.7,2437,Passed,"[Drama, Romance]",1960-09-09,NaN,"[mother son relationship, wilmington delaware,...",NaN,NaN,...,149.0,Mark Robson,/name/nm0733476/,"[John O'Hara, Ernest Lehman]","[/name/nm0641274/, /name/nm0499626/]","[Paul Newman, Joanne Woodward, Myrna Loy]","[/name/nm0000056/, /name/nm0940946/, /name/nm0...",Linebrook,[English],[USA]
1996,V.I. Warshawski,5.0,4713,R,"[Action, Comedy, Crime]",1991-07-26,NaN,"[female stockinged feet, pantyhose, camera sho...",NaN,3603338.0,...,89.0,Jeff Kanew,/name/nm0437596/,"[Sara Paretsky, Edward Taylor, 3 more credits]","[/name/nm0661386/, /name/nm0852324/, fullcredi...","[Kathleen Turner, Jay O. Sanders, Charles Durn...","[/name/nm0000678/, /name/nm0761587/, /name/nm0...",HollywoodPictures,[English],[USA]
1997,Anne of Green Gables,7.0,1114,Passed,"[Comedy, Drama, Family]",1934-11-23,NaN,"[character name as title, anne of green gables...",226000.0,NaN,...,78.0,George Nichols Jr.,/name/nm0629519/,"[Sam Mintz, Lucy Maud Montgomery]","[/name/nm0591791/, /name/nm0599844/]","[Anne Shirley, Tom Brown, O.P. Heggie]","[/name/nm0794297/, /name/nm0114805/, /name/nm0...",RKORadioPictures,[English],[USA]
1998,Odds Against Tomorrow,7.4,3781,Approved,"[Crime, Drama, Thriller]",1959-11-01,NaN,"[new york city, heist, bank robbery, caper, ne...",NaN,NaN,...,100.0,Robert Wise,/name/nm0936404/,"[William P. McGivern, Abraham Polonsky, 2 more...","[/name/nm0569423/, /name/nm0689796/, fullcredi...","[Harry Belafonte, Robert Ryan, Gloria Grahame]","[/name/nm0000896/, /name/nm0752813/, /name/nm0...",HarBelProductions,[English],[USA]


In [235]:
# Create list of actor/link from the 2000 movies
star_list = []
for star in movie_df.star:
    star_list += star
star_df = pd.DataFrame(star_list)

star_link_list = []
for star_link in movie_df.link_s:
    star_link_list += star_link
star_link_df = pd.DataFrame(star_link_list)

# Combine in to one df
star_df['link_s'] = star_link_df
star_df.columns = ['star','link_s']
star_df = star_df.drop_duplicates(subset=['star'])
star_df

,star,link_s
0,Josephine Langford,/name/nm6466214/
1,Hero Fiennes Tiffin,/name/nm2842005/
2,Dylan Sprouse,/name/nm0819851/
3,Marlon Brando,/name/nm0000008/
4,Al Pacino,/name/nm0000199/
...,...,...
5987,Jay O. Sanders,/name/nm0761587/
5990,Tom Brown,/name/nm0114805/
5991,O.P. Heggie,/name/nm0373773/
5992,Harry Belafonte,/name/nm0000896/


In [237]:
star_df.star.nunique(),star_df.link_s.nunique(),

(2893, 2893)

In [ ]:
actor_link_dict = star_df.set_index('star').T.to_dict('list')
actor_link_dict

In [240]:
actor = 'Liam Neeson'
actor_url = base_url + actor_link_dict[actor][0]
actor_url

'https://www.imdb.com/name/nm0000553/'

In [223]:
actor_film_list = []

base_url = "https://www.imdb.com"
c = 0
for actor in actor_link_dict:
    c += 1
    print(c, actor)
    actor_url = base_url + actor_link_dict[actor][0]
    actor_soup = get_soup(actor_url)
    
    # actor or actress
    actor_head = star_soup.find('div', id="filmo-head-actor")
    if actor_head is None:
        actor_head = star_soup.find('div', id="filmo-head-actress")
    
    films = actor_head.findNext('div')

    for film in films:
        movie = film.find('a')
        if type(movie) != int:
            movie_title = movie.text
            movie_link = movie.get('href')
            movie_soup = get_soup(base_url + movie_link)

            detail = get_movie_brief_actor(actor,movie_soup)
            actor_film_list.append(detail)

1 Josephine Langford
2 Hero Fiennes Tiffin
3 Dylan Sprouse
4 Marlon Brando
5 Al Pacino
6 James Caan
7 Daniel Radcliffe
8 Rupert Grint
9 Richard Harris
10 Liam Neeson
11 Diane Kruger
12 January Jones
13 Elijah Wood
14 Ian McKellen
15 Orlando Bloom
16 Marc Singer
17 Tanya Roberts
18 Rip Torn
19 Saoirse Ronan
20 Emma Watson
21 Florence Pugh
22 Tye Sheridan
23 Olivia Cooke
24 Ben Mendelsohn
25 Elisabeth Moss
26 Oliver Jackson-Cohen
27 Harriet Dyer
28 Robert De Niro
29 Ray Liotta
30 Joe Pesci
31 Keira Knightley
32 Matthew Macfadyen
33 Brenda Blethyn
34 Leonardo DiCaprio
35 Tom Hanks
36 Christopher Walken
37 Roman Griffin Davis
38 Thomasin McKenzie
39 Scarlett Johansson
40 Khadijha Red Thunder
41 Robin Wright
42 Gary Sinise
43 Robert Downey Jr.
44 Antonio Banderas
45 Michael Sheen
46 Eric Bana
47 Genevieve O'Reilly
48 Keir O'Donnell
49 Anne Hathaway
50 Octavia Spencer
51 Stanley Tucci
52 Viggo Mortensen
53 Ewan McGregor
54 Rebecca Ferguson
55 Kyliegh Curran
56 Christian Bale
57 Hugh Jackman


457 Ansel Elgort
458 Michael Peña
459 Rhona Mitra
460 Cate Blanchett
461 Sarah Paulson
462 Josh Hartnett
463 Tom Sizemore
464 Jason Schwartzman
465 Sally Ann Howes
466 Lionel Jeffries
467 Ki Hong Lee
468 Dev Patel
469 Freida Pinto
470 Saurabh Shukla
471 Tom Taylor
472 Cobie Smulders
473 Aldis Hodge
474 Ralph Fiennes
475 Charlotte Gainsbourg
476 Lauren Bacall
477 Ingrid Bergman
478 Nat Wolff
479 Luke Wilson
480 Selma Blair
481 Joaquin Phoenix
482 Demi Moore
483 Burt Reynolds
484 Armand Assante
485 Helen Mirren
486 Russell Tovey
487 Adam Driver
488 Joey King
489 Jacob Elordi
490 Joel Courtney
491 Marisa Berenson
492 James Mason
493 Shelley Winters
494 Sue Lyon
495 John Goodman
496 John Cusack
497 Iben Hjejle
498 Todd Louiso
499 Denzel Washington
500 Suraj Sharma
501 Adil Hussain
502 Nick Robinson
503 Jennifer Garner
504 Josh Duhamel
505 Juliette Binoche
506 Tim Roth
507 Edgar Ramírez
508 Joan Allen
509 Nicholas Hoult
510 Toni Collette
511 Marisa Tomei
512 Pam Grier
513 Robert Forster
514

911 John Stockwell
912 Alexandra Paul
913 Ali MacGraw
914 John Marley
915 Timothy Bottoms
916 Cybill Shepherd
917 John Hawkes
918 Garret Dillahunt
919 Jane Alexander
920 Steve Martin
921 Hilary Duff
922 Diana Ross
923 Michael Jackson
924 Nipsey Russell
925 Juno Temple
926 Max Minghella
927 Michael Chiklis
928 Zoey Deutch
929 Halston Sage
930 Cynthy Wu
931 Amber Tamblyn
932 Alexis Bledel
933 William Powell
934 Myrna Loy
935 Maureen O'Sullivan
936 Daniel Baldwin
937 Sheryl Lee
938 Emma Nelson
939 Elvis Presley
940 Marlyn Mason
941 Nicole Jaffe
942 Lucy Fry
943 Danila Kozlovsky
944 Gabriel Byrne
945 Laysla De Oliveira
946 Avery Whitted
947 Hugh Dancy
948 Ron Livingston
949 Christopher Guard
950 William Squire
951 Michael Scholes
952 Jamie Blackley
953 James Cromwell
954 Magda Szubanski
955 Christine Cavanaugh
956 Jessica Tandy
957 Mary Stuart Masterson
958 Vincent Spano
959 Josh Hamilton
960 Stuart Townsend
961 Peta Wilson
962 Diane Venora
963 Dennis Storhøi
964 Robert Arkins
965 Michael 

1348 Simon MacCorkindale
1349 Ewa Aulin
1350 Charles Aznavour
1351 Craig Bierko
1352 Gretchen Mol
1353 Armin Mueller-Stahl
1354 Predrag 'Miki' Manojlovic
1355 Lazar Ristovski
1356 Mirjana Jokovic
1357 Peter Sarsgaard
1358 Jonathan Jackson
1359 Paul Lukas
1360 Polly Bergen
1361 Susan Hayward
1362 Claude Rains
1363 Gloria Stuart
1364 William Harrigan
1365 Fredric March
1366 Rose Hobart
1367 Michael Cera
1368 Kat Dennings
1369 Aaron Yoo
1370 Alison Lohman
1371 Will Geer
1372 Delle Bolton
1373 Lew Ayres
1374 Louis Wolheim
1375 John Wray
1376 John Pankow
1377 Richard Attenborough
1378 Peter Finch
1379 Ann-Margret
1380 Vanity
1381 Haley Joel Osment
1382 Helen Hunt
1383 Barbara Parkins
1384 Patty Duke
1385 Paul Burke
1386 Yvonne Strahovski
1387 Campbell Scott
1388 Mary Beth Hurt
1389 Angourie Rice
1390 Debby Ryan
1391 Kathleen Turner
1392 Chris Massoglia
1393 Salma Hayek
1394 Ella Raines
1395 Ward Bond
1396 Rickie Sorensen
1397 Karl Swenson
1398 Carlos Acosta
1399 Radivoje Bukvic
1400 Anna Fa

1778 Caterina Murino
1779 Mariel Hemingway
1780 Michael Paré
1781 Mason Gamble
1782 Ken Ogata
1783 Masayuki Shionoya
1784 Hiroshi Mikami
1785 Merle Oberon
1786 Gary Bond
1787 Chips Rafferty
1788 Mel Ferrer
1789 Glenn Ford
1790 Jocelyn Brando
1791 Michelle Trachtenberg
1792 Gregory Smith
1793 Jennifer Beals
1794 Susanne Benton
1795 RaéVen Kelly
1796 John Hodiak
1797 Vonetta McGee
1798 Courtney B. Vance
1799 Lynn Redgrave
1800 Michael Sacks
1801 Ron Leibman
1802 Eugene Roche
1803 LisaGay Hamilton
1804 Jennifer Morrison
1805 Nicola Peltz
1806 Ida Lupino
1807 John Garfield
1808 Jonathan Taylor Thomas
1809 Charles Rocket
1810 Joel Grey
1811 Katharine Hepburn
1812 David Selby
1813 Rene Auberjonois
1814 Gabriel Macht
1815 Jim Nabors
1816 David Huffman
1817 Nick Frost
1818 Peter O'Toole
1819 Joanna Going
1820 Gordon Heath
1821 Maurice Denham
1822 Ray Winstone
1823 Bella Thorne
1824 Richard Harmon
1825 Sô Yamamura
1826 Sônia Braga
1827 Annette Bening
1828 Elizabeth Perkins
1829 Richard Hagerman

2212 Johnny Crawford
2213 Guy Madison
2214 Kerwin Mathews
2215 Judi Meredith
2216 Torin Thatcher
2217 David Janssen
2218 Linda Cristal
2219 Al Lettieri
2220 Katy Jurado
2221 Linda Darnell
2222 Ann Sothern
2223 Alan Badel
2224 Burl Ives
2225 Suzanne Pleshette
2226 Charles Ruggles
2227 Liv Ullmann
2228 Sally Kellerman
2229 Hugh Marlowe
2230 Gary Merrill
2231 Sabu
2232 Joseph Calleia
2233 John Qualen
2234 Michael Rennie
2235 Jill St. John
2236 David Hedison
2237 Camilla Sparv
2238 Janet Suzman
2239 Zakes Mokae
2240 Kôji Yakusho
2241 Petula Clark
2242 Michael Redgrave
2243 Dewey Martin
2244 Anne Collings
2245 Stuart Whitman
2246 Ina Balin
2247 Shirley Temple
2248 Richard Greene
2249 Anita Louise
2250 Peter Cushing
2251 Caroline Munro
2252 Elizabeth Allan
2253 Edna May Oliver
2254 Gabe Nevins
2255 Daniel Liu
2256 Taylor Momsen
2257 Jeffrey Wright
2258 Kelly Reilly
2259 Gene Kelly
2260 Martin Shaw
2261 Joss Ackland
2262 Susan Tyrrell
2263 Bruce Bennett
2264 Greta Gerwig
2265 Nina Arianda
226

2643 Elissa Landi
2644 Jack Colvin
2645 Josh Stewart
2646 Candy Clark
2647 Robbie Kay
2648 Matt Lucas
2649 Peter Stormare
2650 William Shatner
2651 Frank Maxwell
2652 Beverly Lunsford
2653 Christine Kaufmann
2654 Leslie Bibb
2655 Raphael Sbarge
2656 Jennifer Runyon
2657 Laura Morante
2658 Juan Diego Botto
2659 Cathy O'Donnell
2660 Paul Muni
2661 Luise Rainer
2662 Walter Connolly
2663 Mark Stevens
2664 Glynis Johns
2665 Cameron Mitchell
2666 Rex Thompson
2667 Rory Calhoun
2668 Ed Morrone
2669 Randy Couture
2670 Drew Pinsky
2671 Richard Conte
2672 Edith Meeks
2673 Larry Maxwell
2674 Susan Norman
2675 Peter Lorre
2676 Frances Drake
2677 Michael Dudikoff
2678 David Carradine
2679 Liza Minnelli
2680 Wendell Burton
2681 Tim McIntire
2682 Michael Ontkean
2683 Joan Taylor
2684 Donald Curtis
2685 Graham Rogers
2686 Stefanie Scott
2687 Frank Silvera
2688 Michele Carey
2689 Don Porter
2690 Ed Quinlan
2691 Jeremy Davies
2692 Amy Locane
2693 Noah Wyle
2694 Jean-Marc Barr
2695 Rubén Blades
2696 Rich

In [224]:
len(actor_film_list)

86790

In [225]:
# Save to pickle
import pickle
with open('../dump/actor_film_list', 'wb') as f:
    pickle.dump(actor_film_list, f)

In [226]:
import pandas as pd
actor_film_list = pd.read_pickle('../dump/actor_film_list')

In [227]:
len(actor_film_list)

86790

In [ ]:
# 1/19/2020 df has problem...
# Found the bug! forgot to change star_soup to actor_soup
# Hence the repeating movies...
actor_film_list2 = []

base_url = "https://www.imdb.com"
c = 0
for actor in actor_link_dict:
    c += 1
    print(c, actor)
    actor_url = base_url + actor_link_dict[actor][0]
    actor_soup = get_soup(actor_url)
    
    # actor or actress
    actor_head = actor_soup.find('div', id="filmo-head-actor")
    if actor_head is None:
        actor_head = actor_soup.find('div', id="filmo-head-actress")
    
    films = actor_head.findNext('div')

    for film in films:
        movie = film.find('a')
        if type(movie) != int:
            movie_title = movie.text
            movie_link = movie.get('href')
            movie_soup = get_soup(base_url + movie_link)

            detail = get_movie_brief_actor(actor,movie_soup)
            actor_film_list2.append(detail)

1 Josephine Langford
2 Hero Fiennes Tiffin
3 Dylan Sprouse
4 Marlon Brando
5 Al Pacino
6 James Caan
7 Daniel Radcliffe
8 Rupert Grint
9 Richard Harris
10 Liam Neeson
11 Diane Kruger
12 January Jones
13 Elijah Wood
14 Ian McKellen
15 Orlando Bloom
16 Marc Singer
17 Tanya Roberts
18 Rip Torn
19 Saoirse Ronan
20 Emma Watson
21 Florence Pugh
22 Tye Sheridan
23 Olivia Cooke
24 Ben Mendelsohn
25 Elisabeth Moss
26 Oliver Jackson-Cohen
27 Harriet Dyer
28 Robert De Niro
29 Ray Liotta
30 Joe Pesci
31 Keira Knightley
32 Matthew Macfadyen
33 Brenda Blethyn
34 Leonardo DiCaprio
35 Tom Hanks
36 Christopher Walken


In [ ]:
# Save to pickle
import pickle
with open('../dump/actor_film_list2', 'wb') as f:
    pickle.dump(actor_film_list2, f)

In [ ]:
# Convert to df
actor_correct_df = pd.DataFrame(actor_film_list2)

In [ ]:
# Save df as pickle
actor_df.to_pickle('../dump/actor_correct_data')

In [229]:
actor_df = pd.DataFrame(actor_film_list)

In [230]:
actor_df

,actor,title,year,rating,vote,genre_list,budget,opening,gross_usa,gross_cw,runtime,director,writer,star,distributor
0,Josephine Langford,Ray Meets Helen,2018-05-04,5.3,151.0,"[Drama, Fantasy, Romance]",NaN,NaN,NaN,NaN,100,Alan Rudolph,[Alan Rudolph],"[Keith Carradine, Sondra Locke, Keith David]",NaN
1,Josephine Langford,The Prophet's Game,2001-04-10,5.0,913.0,"[Crime, Drama, Thriller]",28000000.0,NaN,NaN,NaN,106,David Worth,[Carol Chrest],"[Dennis Hopper, Stephanie Zimbalist, Robert Yo...",MoonstoneEntertainment
2,Josephine Langford,Clean and Narrow,2000-01-21,5.3,129.0,"[Crime, Drama]",2500000.0,NaN,NaN,NaN,82,William Katt,"[Kermit Christman, William Katt]","[Jack Noseworthy, Laura Leighton, Jackie Belvin]",DelMarProductions
3,Josephine Langford,Ratboy,1986-10-17,3.7,599.0,"[Comedy, Crime, Drama]",NaN,NaN,NaN,NaN,104,Sondra Locke,[Rob Thompson],"[Sondra Locke, Robert Townsend, Christopher He...",TheMalpasoCompany
4,Josephine Langford,Amazing,NaT,7.4,5130.0,"[Adventure, Comedy, Drama]",NaN,NaN,NaN,NaN,30,Steven Spielberg,"[Charles Durning, Douglas Seale, Louis Giambal...",NaN,AmblinEntertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86785,Alexis Smith,A Reflection of Fear,1973-05-11,6.0,531.0,"[Crime, Drama, Horror]",NaN,NaN,NaN,NaN,89,William A. Fraker,"[Edward Hume, Lewis John Carlino]","[Robert Shaw, Sally Kellerman, Mary Ure]",ColumbiaPictures
86786,Alexis Smith,Night,NaT,8.0,4770.0,"[Drama, Fantasy, Horror]",NaN,NaN,NaN,NaN,50,Rod Serling,"[Rod Serling, Larry Watson, Joanna Pettet, See...",NaN,UniversalTelevision
86787,Alexis Smith,Willard,1971-07-30,6.2,3735.0,[Horror],NaN,NaN,19216000.0,NaN,95,Daniel Mann,"[Gilbert Ralston, Stephen Gilbert]","[Bruce Davison, Elsa Lanchester, Sondra Locke]",BingCrosbyProductions
86788,Alexis Smith,Cover Me Babe,1970-10-01,4.4,154.0,[Drama],NaN,NaN,NaN,NaN,86,Noel Black,[George Wells],"[Robert Forster, Sondra Locke, Susanne Benton]",TwentiethCenturyFox


In [231]:
# Save df as pickle
actor_df.to_pickle('../dump/actor_data')

In [232]:
# Test load pickle
actor_df = pd.read_pickle('../dump/actor_data')

In [233]:
actor_df

,actor,title,year,rating,vote,genre_list,budget,opening,gross_usa,gross_cw,runtime,director,writer,star,distributor
0,Josephine Langford,Ray Meets Helen,2018-05-04,5.3,151.0,"[Drama, Fantasy, Romance]",NaN,NaN,NaN,NaN,100,Alan Rudolph,[Alan Rudolph],"[Keith Carradine, Sondra Locke, Keith David]",NaN
1,Josephine Langford,The Prophet's Game,2001-04-10,5.0,913.0,"[Crime, Drama, Thriller]",28000000.0,NaN,NaN,NaN,106,David Worth,[Carol Chrest],"[Dennis Hopper, Stephanie Zimbalist, Robert Yo...",MoonstoneEntertainment
2,Josephine Langford,Clean and Narrow,2000-01-21,5.3,129.0,"[Crime, Drama]",2500000.0,NaN,NaN,NaN,82,William Katt,"[Kermit Christman, William Katt]","[Jack Noseworthy, Laura Leighton, Jackie Belvin]",DelMarProductions
3,Josephine Langford,Ratboy,1986-10-17,3.7,599.0,"[Comedy, Crime, Drama]",NaN,NaN,NaN,NaN,104,Sondra Locke,[Rob Thompson],"[Sondra Locke, Robert Townsend, Christopher He...",TheMalpasoCompany
4,Josephine Langford,Amazing,NaT,7.4,5130.0,"[Adventure, Comedy, Drama]",NaN,NaN,NaN,NaN,30,Steven Spielberg,"[Charles Durning, Douglas Seale, Louis Giambal...",NaN,AmblinEntertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86785,Alexis Smith,A Reflection of Fear,1973-05-11,6.0,531.0,"[Crime, Drama, Horror]",NaN,NaN,NaN,NaN,89,William A. Fraker,"[Edward Hume, Lewis John Carlino]","[Robert Shaw, Sally Kellerman, Mary Ure]",ColumbiaPictures
86786,Alexis Smith,Night,NaT,8.0,4770.0,"[Drama, Fantasy, Horror]",NaN,NaN,NaN,NaN,50,Rod Serling,"[Rod Serling, Larry Watson, Joanna Pettet, See...",NaN,UniversalTelevision
86787,Alexis Smith,Willard,1971-07-30,6.2,3735.0,[Horror],NaN,NaN,19216000.0,NaN,95,Daniel Mann,"[Gilbert Ralston, Stephen Gilbert]","[Bruce Davison, Elsa Lanchester, Sondra Locke]",BingCrosbyProductions
86788,Alexis Smith,Cover Me Babe,1970-10-01,4.4,154.0,[Drama],NaN,NaN,NaN,NaN,86,Noel Black,[George Wells],"[Robert Forster, Sondra Locke, Susanne Benton]",TwentiethCenturyFox


In [242]:
actor_df.actor.nunique(),actor_df.title.nunique()

(2893, 30)

In [179]:
left_actor_set = star_list_set.difference(set(done_actor))

In [180]:
len(left_actor_set)

2893

In [50]:
# actor = "Zooey Deschanel"
star_soup = get_soup(star_url)
base_url = "https://www.imdb.com"

actor_film_list=[]
# actor name
actor = star_soup.find('h1').text.strip('\n').lstrip(' ')

# actor or actress
actor_head = star_soup.find('div', id="filmo-head-actor")
if actor_head is None:
    actor_head = star_soup.find('div', id="filmo-head-actress")
films = actor_head.findNext('div')

for film in films:
    movie = film.find('a')
    if type(movie) != int:
        movie_title = movie.text
        movie_link = movie.get('href')
        movie_soup = get_soup(base_url + movie_link)

        detail = get_movie_brief_actor(actor,movie_soup)
        actor_film_list.append(detail)

In [48]:
star_soup.find('h1').text.strip('\n').lstrip(' ')

'Zooey Deschanel'

In [57]:
# actor = "George Clooney"
star_url = 'https://www.imdb.com/name/nm0000123/'
star_soup = get_soup(star_url)
base_url = "https://www.imdb.com"


# actor name
actor = star_soup.find('h1').text.strip('\n').lstrip(' ')

# actor or actress
actor_head = star_soup.find('div', id="filmo-head-actor")
if actor_head is None:
    actor_head = star_soup.find('div', id="filmo-head-actress")
films = actor_head.findNext('div')

for film in films:
    movie = film.find('a')
    if type(movie) != int:
        movie_title = movie.text
        movie_link = movie.get('href')
        movie_soup = get_soup(base_url + movie_link)

        detail = get_movie_brief_actor(actor,movie_soup)
        actor_film_list.append(detail)

In [191]:
actor_film_list

[{'actor': 'Sondra Locke\n(1944–2018)',
  'title': 'Ray Meets Helen',
  'year': Timestamp('2018-05-04 00:00:00'),
  'rating': 5.3,
  'vote': 151,
  'genre_list': ['Drama', 'Fantasy', 'Romance'],
  'budget': nan,
  'opening': nan,
  'gross_usa': nan,
  'gross_cw': nan,
  'runtime': 100,
  'director': 'Alan Rudolph',
  'writer': ['Alan Rudolph'],
  'star': ['Keith Carradine', 'Sondra Locke', 'Keith David'],
  'distributor': nan},
 {'actor': 'Sondra Locke\n(1944–2018)',
  'title': "The Prophet's Game",
  'year': Timestamp('2001-04-10 00:00:00'),
  'rating': 5.0,
  'vote': 913,
  'genre_list': ['Crime', 'Drama', 'Thriller'],
  'budget': 28000000,
  'opening': nan,
  'gross_usa': nan,
  'gross_cw': nan,
  'runtime': 106,
  'director': 'David Worth',
  'writer': ['Carol Chrest'],
  'star': ['Dennis Hopper', 'Stephanie Zimbalist', 'Robert Yocum'],
  'distributor': 'MoonstoneEntertainment'},
 {'actor': 'Sondra Locke\n(1944–2018)',
  'title': 'Clean and Narrow',
  'year': Timestamp('2000-01-21 